In [1]:
from captum.attr import LayerIntegratedGradients, IntegratedGradients
from collections import OrderedDict
from transformers import XLNetTokenizer
import torch

In [2]:
from modified_xlnet import XLNetForSequenceClassification

In [3]:
model_path = "/scratch/kh8fb/2020summer_research/trained_models/xlnet_imdb/imdb-finetuned-xlnet-model_1.pth"
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased")
model_states = torch.load(model_path, map_location=torch.device('cpu'))
new_model_states = OrderedDict()
for state in model_states:
    correct_state = state[7:]
    new_model_states[correct_state] = model_states[state]
model.load_state_dict(new_model_states)
model.eval()
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [4]:
sentence = "might not find it to totally be like Pokémon without Ash."                                                                                               
sentence2 = "But this is definitely a Pokémon movie and way better than most of their animated movies."                                                                                      

features = tokenizer([sentence, sentence2], return_tensors='pt', padding=True, truncation=True, max_length=512)

input_ids = features["input_ids"] #.to("cuda:0")
token_type_ids = features["token_type_ids"] #.to("cuda:0")
attention_mask = features["attention_mask"] #.to("cuda:0")
baseline_ids = torch.zeros(input_ids.shape, dtype=torch.int64, device="cpu") #, device="cuda:0")

In [5]:
def sequence_forward_func(inputs, model, tok_type_ids, att_mask):
    """                                                                                                                                                                            
    Passes forward the inputs and relevant keyword arguments.                                                                                                                      
                                                                                                                                                                                   
    Parameters                                                                                                                                                                     
    ----------                                                                                                                                                                     
    inputs: torch.tensor(1, num_ids), dtype=torch.int64                                                                                                                            
        Encoded form of the input sentence.                                                                                                                                        
    tok_type_ids: torch.tensor(1, num_ids), dtype=torch.int64                                                                                                                      
        Tensor to specify token type for the model.                                                                                                                                
        Because sentiment analysis uses only one input, this is just a tensor of zeros.                                                                                            
    att_mask: torch.tensor(1, num_ids), dtype=torch.int64                                                                                                                          
        Tensor to specify attention masking for the model.                                                                                                                         
                                                                                                                                                                                   
    Returns                                                                                                                                                                        
    -------                                                                                                                                                                        
    outputs: torch.tensor(1, 2), dtype=torch.float32                                                                                                                               
        Output classifications for the model.                                                                                                                                      
    """
    print("running Output")
    print("Inputs shape: ", inputs.shape)
    print("token type shape:", tok_type_ids.shape)
    print("att mask shape: ", att_mask.shape)
    outputs = model(input_ids=inputs, token_type_ids=tok_type_ids, attention_mask=att_mask)
    print(outputs[0])
    return outputs

In [6]:
import numpy as np
rand_inputs = torch.tensor(np.random.randint(0, 3000, size=(100, 20)), dtype=torch.int64)
model.forward(rand_inputs)

Batch size:  100
input ids shape: torch.Size([20, 100])
self.word_embedding(input_ids) shape:  torch.Size([20, 100, 768])
word embedding type:  <class 'torch.nn.modules.sparse.Embedding'>
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 100, 

(tensor([[-0.5426, -0.4964],
         [-0.7126, -0.0405],
         [-0.0765, -0.9918],
         [-0.7048, -0.1508],
         [-0.4918, -0.5851],
         [-0.4771, -0.3674],
         [-0.3369, -0.5912],
         [-0.3918, -0.7830],
         [-0.5138, -0.3121],
         [-0.6954, -0.2804],
         [-1.0564, -0.1028],
         [-0.3149, -0.5230],
         [-0.7914, -0.3071],
         [-0.2181, -0.4175],
         [-0.4536, -0.3613],
         [-0.4210, -0.7020],
         [-0.4556, -0.2807],
         [-0.0180, -1.0441],
         [-0.8096, -0.0700],
         [-0.6632, -0.2860],
         [-0.1843, -0.7645],
         [-0.2577, -0.3888],
         [-0.6528, -0.2826],
         [-0.1008, -0.8373],
         [-1.0375,  0.0479],
         [-0.3798, -0.6566],
         [-0.3688, -0.3710],
         [-0.6740, -0.5063],
         [-0.3206, -0.5344],
         [ 0.1459, -1.1453],
         [-0.8059,  0.0320],
         [-0.7673, -0.1948],
         [-0.4214, -0.6159],
         [-0.3603, -0.4546],
         [-1.1

In [ ]:
model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

In [7]:
#model = model.to('cuda:0')
layer_integrated = LayerIntegratedGradients(sequence_forward_func, model.transformer.word_embedding)

In [8]:
attrs = layer_integrated.attribute(inputs=input_ids,
                                     baselines=baseline_ids,
                                     additional_forward_args=(model, token_type_ids, attention_mask),
                                     n_steps=50,
                                     target=0,
                                     return_convergence_delta=False)

captum inputs shape
torch.Size([2, 20])
torch.Size([2, 20])
torch.Size([2, 20])
forward_func args:  OrderedDict([('inputs', <Parameter "inputs">), ('model', <Parameter "model">), ('tok_type_ids', <Parameter "tok_type_ids">), ('att_mask', <Parameter "att_mask">)])
running Output
Inputs shape:  torch.Size([2, 20])
token type shape: torch.Size([2, 20])
att mask shape:  torch.Size([2, 20])
Batch size:  2
input ids shape: torch.Size([20, 2])
EVAL TENSORS
1
self.word_embedding(input_ids) shape:  torch.Size([20, 2, 768])
word embedding type:  <class 'torch.nn.modules.sparse.Embedding'>
EVAL TENSORS
1
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g shape: None
('Output_h shape: ', torch.Size([20, 2, 768]))
Output_g sha

RuntimeError: size of dimension does not match previous size, operand 1, dim 1